In [1]:
#add shift and model parameters based on basic model

from pyspark.sql import SparkSession

# supress warnings
import warnings
warnings.filterwarnings('ignore')

spark = SparkSession.builder \
    .appName("Presidential Election Prediction") \
    .getOrCreate()

df = spark.read.parquet("part-00000-4fa0b8fe-f36e-45eb-91d1-c61026c3c0cd-c000.snappy.parquet")
df.printSchema()
df.show()  

24/04/24 15:07:55 WARN Utils: Your hostname, yangjingdeMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.7 instead (on interface en0)
24/04/24 15:07:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/24 15:07:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- file_name: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: integer (nullable = true)

+--------------------+--------------------+-----+
|           file_name|            features|label|
+--------------------+--------------------+-----+
|file:///Users/Son...|[-11.0,-0.0045544...|    0|
|file:///Users/Son...|[-4.0,-0.00696119...|    1|
|file:///Users/Son...|[6.0,-0.004067219...|    1|
|file:///Users/Son...|[2.0,-0.004230739...|    1|
|file:///Users/Son...|[-16.0,-0.0048420...|    1|
|file:///Users/Son...|[-1.0,-0.00354826...|    0|
|file:///Users/Son...|[-3.0,-0.00778278...|    1|
|file:///Users/Son...|[-1.0,-0.00144967...|    0|
|file:///Users/Son...|[-15.0,-0.0018874...|    1|
|file:///Users/Son...|[-4.0,6.327797309...|    1|
|file:///Users/Son...|[-2.0,-0.00541289...|    1|
|file:///Users/Son...|[14.0,-0.00546586...|    0|
|file:///Users/Son...|[3.0,-0.003829947...|    1|
|file:///Users/Son...|[-4.0,-0.00344390...|    0|
|file:///Users/Son...|[-7

In [2]:
df.dtypes

[('file_name', 'string'), ('features', 'vector'), ('label', 'int')]

In [3]:
from pyspark.ml.classification import NaiveBayes, LogisticRegression, LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import MinMaxScaler

In [4]:
# Handle Negative Values: since our features columns contains negative values, we use scaling techniques such as MinMaxScaler or StandardScaler to transform or standarize it.
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(df)
df = scaler_model.transform(df)

In [5]:
# Split Data into Training and Testing Sets
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Train the models(NaiveBayes,LogisticRegression,SVM)
lr = LogisticRegression(featuresCol="scaled_features", labelCol="label")
svm = LinearSVC(featuresCol="scaled_features", labelCol="label")
nb = NaiveBayes(featuresCol="scaled_features", labelCol="label")

lr_model = lr.fit(train_data)
svm_model = svm.fit(train_data)
nb_model = nb.fit(train_data)

24/04/24 15:08:00 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/04/24 15:08:00 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [6]:
# Model Evaluation (using accuracy as an example)
lr_predictions = lr_model.transform(test_data)
svm_predictions = svm_model.transform(test_data)
nb_predictions = nb_model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

lr_accuracy = evaluator.evaluate(lr_predictions)
svm_accuracy = evaluator.evaluate(svm_predictions)
nb_accuracy = evaluator.evaluate(nb_predictions)

print("Logistic Regression Accuracy:", lr_accuracy)
print("SVM Accuracy:", svm_accuracy)
print("Naive Bayes Accuracy:", nb_accuracy)

Logistic Regression Accuracy: 0.36363636363636365
SVM Accuracy: 0.5
Naive Bayes Accuracy: 0.5909090909090909
